In [ ]:
import xarray as xr
import numpy as np
import cftime

In [ ]:
start = '2010-01-01'
end = '2020-12-31'

# Create noleap datarray
times = xr.cftime_range(start=start, end=end, freq='D', calendar='noleap')
data = xr.DataArray(np.arange(times.size * 4).reshape(times.size, 2, 2) ,dims=['time', 'lat', 'lon'], coords={'time': times})
data

In [ ]:
times_2 = xr.DataArray(xr.cftime_range(start=start, end=end, freq='D'),dims=['time'])
times_2

In [ ]:
cond = (times_2.time.dt.month == 2) & (times_2.time.dt.day == 29)

In [ ]:
leap_days = times_2.where(cond, drop=True)
leap_days

In [ ]:
def convert(time_obj):
    converted = cftime.DatetimeGregorian(year=time_obj.year, month=time_obj.month, day=time_obj.day)
    return converted


times_greg = [convert(time_obj) for time_obj in data.time.data]

In [ ]:
data["time"] = times_greg

In [ ]:
data

In [ ]:
one_time_step = data.isel(time=slice(0, 1))
one_time_step

In [ ]:
x = []
for leap_day in leap_days:
    d = xr.full_like(one_time_step,fill_value=np.nan)
    d = d.assign_coords(time=[leap_day.data])
    x.append(d)

In [ ]:
x[0]

In [ ]:
x[1]

In [ ]:
data

In [ ]:
x.append(data)

ds=xr.concat(x, dim='time').sortby('time')
ds